<a href="https://colab.research.google.com/github/KairaNithin/XML-Experiments/blob/main/paper_XLM__freeze_multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install sentence_transformers
!pip install transformers
from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 81kB 3.7MB/s 
     |████████████████████████████████| 2.3MB 9.7MB/s 
     |████████████████████████████████| 1.2MB 36.2MB/s 
     |████████████████████████████████| 901kB 31.3MB/s 
     |████████████████████████████████| 3.3MB 32.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=6b82313560f8d7f48fe8d40a4910c4ebf4921a1aa52d1e297a83d63abac43771
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [8]:
import torch
if torch.cuda.is_available():
  print("There are %d Gpu's available"%torch.cuda.device_count())
  print("using the {} GPU".format(torch.cuda.get_device_name(0)))
else :
  print("No Gpu's available, using the cpu instead")

No Gpu's available, using the cpu instead


In [12]:
import random
import pickle
import re
import time
import datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
# Check where we need this
# from nltk.corpus import stopwordsm
from nltk.tokenize import sent_tokenize

import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import Dataset
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelWithLMHead


import torch.nn as nn
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import gensim.models as gsm

from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


from tqdm import tqdm 
import gc
import os

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
train_dataloader, valid_dataloader, dataloader, dataloaders = torch.load('drive/MyDrive/train_dataloader.pth'), torch.load('drive/MyDrive/valid_dataloader.pth'),torch.load('drive/MyDrive/dataloader.pth'),torch.load('drive/MyDrive/dataloaders.pth')

In [13]:
class ClassificationHead(nn.Module):
  """ Classification head for the Roberta Model """ 
  def __init__(self, numberOfClasses, hidden_size_bert, hidden_size_post_feats, dropout_val = 0.2):
    super().__init__()
    self.denseInit = nn.Linear(hidden_size_post_feats, hidden_size_bert)
    self.dense = nn.Linear(hidden_size_bert, hidden_size_bert)
    self.dropout = nn.Dropout(dropout_val)
    self.output = nn.Linear(hidden_size_bert, numberOfClasses)
  def forward(self, x):
    # print(x.shape)
    x = self.dropout(x)
    x = self.denseInit(x)
    x = torch.tanh(x)
    x = self.dropout(x)
    x = self.dense(x)
    x  = torch.tanh(x)
    x = self.dropout(x)
    x  = self.output(x)
    return x

class TextClassification(nn.Module):
  """ Classifier with feature injection """
  def __init__(self, numberOfClasses,dropout_val = 0.1, batch_size = 16):
     super(TextClassification, self).__init__()
     self.bert = XLMRobertaModel.from_pretrained("sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens")
     self.classifier = ClassificationHead(numberOfClasses, self.bert.config.hidden_size, (self.bert.config.hidden_size * 2 + 300) , dropout_val)
  def forward(self, input_seq, attention_mask, emoji, hashTag):
    bert_pooled_output = self.bert(input_seq, attention_mask=attention_mask)[0]
    bert_pooled_output = bert_pooled_output[:, 0, :]
    bert_pooled_out_feat = torch.cat([bert_pooled_output, emoji, hashTag], axis = 1)
    # print("Shape",bert_pooled_out_feat.shape)
    output = self.classifier(bert_pooled_out_feat)
    return output

In [14]:
model_name = 'adaptive'
model_loc = '/content/drive/My Drive/hasoc_saved/'

In [15]:
def modelEvaluate(model, valid_dataloader = valid_dataloader, task = 1):
  gc.collect()
  if task == 1:
    taskIndex = 6
  elif task == 2:
    taskIndex = 7
  model.eval()
  predictions, true_labels = [], []
  logits = []
  # Predict 
  for batch in valid_dataloader:
    # Add batch to GPU
    b_input_ids = batch[1]
    b_input_mask = batch[2]
    b_labels = batch[taskIndex]
    b_emoji = batch[5]
    b_hashtag = batch[4]
    with torch.no_grad():
      pred = model(b_input_ids,b_input_mask ,b_emoji.float(), b_hashtag.float())
    logits.append(pred.detach().cpu().numpy())
    label_ids = b_labels.to('cpu').numpy()
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)
    flat_true_labels = np.concatenate(true_labels, axis = 0)
    predictions = []
    for i in logits:
      for j in i:
        predictions.append(j)
    flat_predictions = [np.argmax(i) for i in predictions]
    assert(len(flat_predictions) == len(flat_true_labels))
    return flat_predictions, flat_true_labels

In [16]:
path =  model_name + ".pt"
scale = 1

In [32]:
def make_optim(model, rate = 2e-5):
  return AdamW(model.parameters(),
                lr = rate, # default = 5e-5, using 2e-5
                eps = 1e-8) # default = 1e-8

def train_model(train_dataloader, valid_dataloader, numberOfEpochs = 10, task = 1):
  """ Train Loop for the model """
  scale = 1
  if task == 2:
    classNum = 4
    taskIndex = 7
  elif task == 1:
    classNum = 2
    taskIndex = 6
  else:
    raise NameError("Task not defined")
  total_steps = len(train_dataloader)
  print("Start")

  model = TextClassification(classNum) # task 1 
  for params in model.bert.parameters():
    params.requires_grad = False
  if torch.device == "gpu":
    model.cuda()
  device = torch.device("cuda")
  
  loss_function = nn.CrossEntropyLoss().to(device)
  epoch_loss = 0
  batch_accuracy_scores = []
  global_pred = []
  global_label = []

  present_rate = 2e-5
  old_best = -1
  epoch = 0

  while(1):
    # when the learn rate falls below a lower threshold, you stop your training
    # until that moment, march on
    epoch += 1
    print("\nEpoch:", epoch)
    print("Present Rate: " + str(present_rate))
    optimizer = make_optim(model, present_rate)
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)
    gc.collect()
    model.train()
    epoch_loss = 0
    batch_accuracy_scores = []
    train_data_count = float(len(train_dataloader))

    # to check if performance with default weights
    predictions, true_labels = modelEvaluate(model, valid_dataloader, task)
    score_now = f1_score(true_labels, predictions, average = 'macro')
    print("Validation Macro: " + str(score_now))

    if (score_now > old_best):
      print("Continuing on track")
      old_best = score_now

      # delete previous best 
      delete_filename = path
      open(delete_filename, 'w').close() # overwrite and make the file blank instead
      os.remove(delete_filename) # delete the blank file from google drive will move the file to bin instead
      torch.save(model.state_dict(), path)

    else:
      print("Backtrack")
      model.load_state_dict(torch.load(path))
      present_rate /= (4 * scale)
      scale *= 4
      if present_rate < 1e-8:
        break

    # For quick eval
    cnt = 0
    # for i, batch in tqdm(enumerate(train_dataloader)):
    for i, batch in enumerate(train_dataloader):
        print("Iter: " + str(cnt + 1))
        # COMMENT OUT THE NEXT 2 LINES IN ACTUAL TRAINING
        # if cnt == 4:
        #   break
        cnt += 1
        b_input_ids = batch[1]
        b_input_mask = batch[2]
        b_labels = batch[taskIndex]
        b_emoji = batch[5]
        b_hashtag = batch[4]
        pred = model(b_input_ids,b_input_mask ,b_emoji.float(), b_hashtag.float())
        loss = loss_function(pred.view(-1, classNum), b_labels.view(-1))
        with torch.no_grad():
          epoch_loss += (loss.item() * len(b_labels))
          global_pred.append(pred)
          global_label.append(b_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()

  return model

In [33]:
gc.collect()
model = train_model(train_dataloader, valid_dataloader, 2, task = 1)

Start

Epoch: 1
Present Rate: 2e-05
Validation Macro: 0.30980392156862746
Continuing on track
Iter: 1
Iter: 2
Iter: 3
Iter: 4
Iter: 5
Iter: 6
Iter: 7
Iter: 8
Iter: 9
Iter: 10
Iter: 11
Iter: 12
Iter: 13
Iter: 14
Iter: 15
Iter: 16
Iter: 17
Iter: 18
Iter: 19
Iter: 20
Iter: 21
Iter: 22
Iter: 23
Iter: 24
Iter: 25
Iter: 26
Iter: 27
Iter: 28
Iter: 29
Iter: 30
Iter: 31
Iter: 32
Iter: 33
Iter: 34
Iter: 35
Iter: 36
Iter: 37
Iter: 38
Iter: 39
Iter: 40
Iter: 41
Iter: 42
Iter: 43
Iter: 44
Iter: 45
Iter: 46
Iter: 47
Iter: 48
Iter: 49
Iter: 50
Iter: 51
Iter: 52
Iter: 53
Iter: 54
Iter: 55
Iter: 56
Iter: 57
Iter: 58
Iter: 59
Iter: 60
Iter: 61
Iter: 62
Iter: 63
Iter: 64
Iter: 65
Iter: 66
Iter: 67
Iter: 68
Iter: 69
Iter: 70
Iter: 71
Iter: 72
Iter: 73
Iter: 74
Iter: 75
Iter: 76
Iter: 77
Iter: 78
Iter: 79
Iter: 80
Iter: 81
Iter: 82
Iter: 83
Iter: 84
Iter: 85
Iter: 86
Iter: 87
Iter: 88
Iter: 89
Iter: 90
Iter: 91
Iter: 92
Iter: 93
Iter: 94
Iter: 95
Iter: 96
Iter: 97
Iter: 98
Iter: 99
Iter: 100
Iter: 101
Iter

In [39]:
test_loaders = torch.load("drive/MyDrive/XLM_Data/XLM_all_lang_test.pt")

In [40]:
# for hindi
predictions, true_labels = modelEvaluate(model, test_loaders['hi_test_loader'], 1)
f1_score(true_labels, predictions, average='macro')

0.5897435897435896

In [43]:
#for english
predictions, true_labels = modelEvaluate(model, test_loaders['en_test_loader'], 1)
f1_score(true_labels, predictions, average='macro')

0.6536796536796536

In [42]:
#for german
predictions, true_labels = modelEvaluate(model, test_loaders['ge_test_loader'], 1)
f1_score(true_labels, predictions, average='macro')

0.8160919540229885

In [38]:
torch.save(model,"drive/MyDrive/XLM_freeze_multi_adaptive.pt")

In [ ]:
""